# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`toorandmed_aasta.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../andmed/toorandmed_aasta.csv`

**Väljund:** `../andmed/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [1]:
import pandas as pd
import json
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = '../andmed/toorandmed_aasta.csv'
OUTPUT_FILE = '../andmed/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile


print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

df.head(2)

Toorandmed loetud. Andmestiku suurus: 3416 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,state__et,state__en,last_update,title__et,general__input_languages,general__structural_unit_shares,general__type__code,general__type__et,general__type__en,general__structural_units,additional_info__duration_in_semesters,additional_info__assessment_scale__code,additional_info__assessment_scale__et,additional_info__assessment_scale__en,additional_info__is_vota_course,additional_info__is_continuous_learning_course,overview__description__et,overview__objectives,overview__learning_outcomes,_actions__is_editable,_actions__is_version_creatable,_actions__is_deletable,overview__objectives_text_en,version__uuid,version__code,version__state__code,version__state__et,version__state__en,version__last_update,version__title__et,version__credits,version__credit_type,version__target__year__code,version__target__year__et,version__target__year__en,version__target__semester__code,version__target__semester__et,version__target__semester__en,version__target__study_type__code,version__target__study_type__et,version__target__study_type__en,version__target__language__code,version__target__language__et,version__target__language__en,version__target__part__code,version__target__part__et,version__target__part__en,version__target__faculty__code,version__target__faculty__name__en,version__target__faculty__name__et,version__target__faculty__address,version__target__faculty__city,version__target__faculty__level,version__target__faculty__academic,version__target__faculty__supports_continuous_learning,version__target__course_main_structural_unit__code,version__target__course_main_structural_unit__name__en,version__target__course_main_structural_unit__name__et,version__target__course_main_structural_unit__webpage_url,version__target__course_main_structural_unit__address,version__target__course_main_structural_unit__city,version__target__course_main_structural_unit__street,version__target__course_main_structural_unit__zip,version__target__course_main_structural_unit__level,version__target__course_main_structural_unit__academic,version__target__course_main_structural_unit__supports_continuous_learning,version__target__course_main_structural_unit__parent_code,version__target__label__en,version__target__label__et,version__general__year__code,version__general__year__et,version__general__year__en,version__general__input_languages,version__general__structural_unit_shares,version__general__type__code,version__general__type__et,version__general__type__en,version__additional_info__is_vota_course,version__additional_info__is_continuous_learning_course,version__additional_info__study_levels,version__additional_info__hours__lecture,version__additional_info__hours__practice,version__additional_info__hours__seminar,version__additional_info__hours__colloquium,version__additional_info__hours__web_learning,version__additional_info__hours__independent_work,version__additional_info__hours__individual_work,version__additional_info__hours__internship,version__additional_info__is_enlight_course,version__overview__study_languages,version__overview__description__et,version__overview__objectives,version__overview__learning_outcomes,version__grading__independent_work_assessments,version__grading__grade_preconditions__et,version__grading__grade_evaluation__et,version__grading__debt_elimination__et,version__grading__assessment_scale__code,version__grading__assessment_scale__et,version__grading__assessment_scale__en,version__resources__web_based__code,version__resources__web_based__et,version__resources__web_based__en,version__resources__website_url,version__resources__is_moodle_connected,version__resources__learning_environments,version__resources__materials,version__participants__lecturers,version__schedule__weeks__et,version__schedule__entries,version__parent_uuid,version__parent_code,version__parent_credits,version___action

### Samm 1: eemaldame andmed, mida me ei taha

Rakendame minimaalsed eelfiltrid, mis olid seminaris kokku lepitud:
- eemaldame ained, mille kestus on üle 1 semestri;
- eemaldame kaitsmisega seotud ained (kui pealkiri viitab kaitsmisele);
- proovime jätta ainult päevaõppe, kui vastav tunnus on olemas.

### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [2]:
# --- Samm 1: Eelfiltrid ---
df_work = df.copy()
rows_before = len(df_work)

# 1) eemaldame > 1 semestri ained (kui veerg olemas)
if 'additional_info__duration_in_semesters' in df_work.columns:
    duration = pd.to_numeric(df_work['additional_info__duration_in_semesters'], errors='coerce')
    df_work = df_work[(duration.isna()) | (duration <= 1)]

# 2) eemaldame kaitsmisega seotud ained (lihtne heuristika pealkirja põhjal)
title_col = 'version__title__et' if 'version__title__et' in df_work.columns else 'title__et'
if title_col in df_work.columns:
    title_text = df_work[title_col].fillna('').astype(str)
    df_work = df_work[~title_text.str.contains(r'kaitsm', case=False, regex=True)]

# 3) proovime jätta ainult päevaõppe (kui tunnus olemas)
if 'version__target__study_type__et' in df_work.columns:
    study_type = df_work['version__target__study_type__et'].fillna('').astype(str)
    day_mask = study_type.str.contains('päeva', case=False, regex=False)
    if day_mask.any():
        df_work = df_work[day_mask]

print(f"Samm 1: Eelfiltrid rakendatud. Ridasi enne: {rows_before}, pärast: {len(df_work)}")


def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """

    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),
        ('kirjeldus', 'overview__description__et', 'version__overview__description__et'),
        ('eesmargid', 'overview__objectives_text_et', 'version__overview__objectives_text_et'),
        ('opivaljundid', 'overview__learning_outcomes_text_et', 'version__overview__learning_outcomes_text_et'),
        ('hindamisskaala', 'additional_info__assessment_scale__et', 'version__grading__assessment_scale__et'),
    ]

    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('oppekeel', 'version__target__language__et'),
        ('oppeaste', 'version__additional_info__study_levels'),
        ('teaduskond', 'version__target__faculty__name__et'),
        ('instituut', 'version__target__course_main_structural_unit__name__et'),
        ('linn', 'version__target__faculty__city'),
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries'),
    ]

    print("Alustan veergude filtreerimist ja ühendamist...")

    for new_col, base, version in merge_mapping:
        base_exists = bool(base) and (base in df.columns)
        ver_exists = bool(version) and (version in df.columns)

        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan

    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan

    return df


# Rakendame funktsiooni
df_resolved = resolve_fields(df_work.copy())

# Kontrollime tulemust
print("Samm 2: Puhastatud veergude lisamine tehtud.")
print(df_resolved[['aine_kood', 'semester', 'oppekeel', 'oppeaste']].isnull().sum())
print("Semestrid:", sorted([str(x) for x in df_resolved['semester'].dropna().unique()][:10]))

Samm 1: Eelfiltrid rakendatud. Ridasi enne: 3416, pärast: 2940
Alustan veergude filtreerimist ja ühendamist...
Samm 2: Puhastatud veergude lisamine tehtud.
aine_kood     0
semester      0
oppekeel      0
oppeaste     35
dtype: int64
Semestrid: ['kevad', 'sügis']



### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [3]:
# --- Abifunktsioonid JSON töötlemiseks ---

def parse_json_safe(value):
    """Teisendab JSON-stringi turvaliselt Pythoni objektiks (list või dict)."""
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None
    if isinstance(value, (list, dict)):
        return value

    text = str(value).strip()
    if text == '' or text.lower() == 'nan':
        return None
    try:
        return json.loads(text)
    except (json.JSONDecodeError, TypeError):
        return None


def extract_lecturers(json_value):
    data = parse_json_safe(json_value)
    if not data:
        return np.nan
    if isinstance(data, dict):
        data = [data]
    names = [p.get('person_name') for p in data if isinstance(p, dict) and p.get('person_name')]
    return ", ".join(sorted(set(names))) if names else np.nan


def extract_schedule_days_et(json_value):
    data = parse_json_safe(json_value)
    if not data:
        return np.nan
    if isinstance(data, dict):
        data = [data]

    days = set()
    day_map = {
        'Monday': 'Esmaspäev', 'Tuesday': 'Teisipäev', 'Wednesday': 'Kolmapäev',
        'Thursday': 'Neljapäev', 'Friday': 'Reede', 'Saturday': 'Laupäev', 'Sunday': 'Pühapäev'
    }

    for entry in data:
        if not isinstance(entry, dict):
            continue
        time_str = entry.get('time') or entry.get('start_time')
        if time_str:
            dt = pd.to_datetime(time_str, errors='coerce')
            if not pd.isna(dt):
                days.add(day_map.get(dt.day_name(), dt.day_name()))

    week_order = ['Esmaspäev', 'Teisipäev', 'Kolmapäev', 'Neljapäev', 'Reede', 'Laupäev', 'Pühapäev']
    sorted_days = sorted(days, key=lambda d: week_order.index(d) if d in week_order else 99)
    return ", ".join(sorted_days) if sorted_days else np.nan


def extract_text_values(value):
    """Ekstraheerib list/dict/JSON väljast loetava teksti."""
    data = parse_json_safe(value)
    if data is None:
        return value if (pd.notna(value) and str(value).strip() != '') else np.nan

    if isinstance(data, dict):
        data = [data]

    out = []
    if isinstance(data, list):
        for item in data:
            if isinstance(item, str):
                if item.strip():
                    out.append(item.strip())
            elif isinstance(item, dict):
                candidate = (
                    item.get('et')
                    or item.get('name__et')
                    or item.get('name')
                    or item.get('title')
                    or item.get('value')
                    or item.get('code')
                )
                if candidate and str(candidate).strip():
                    out.append(str(candidate).strip())

    unique_vals = []
    for x in out:
        if x not in unique_vals:
            unique_vals.append(x)

    return ", ".join(unique_vals) if unique_vals else np.nan


# --- Funktsioonide rakendamine ---
print("Ekstraheerime JSON väljadest infot...")

df_resolved['oppejoud'] = df_resolved['oppejoud_json'].apply(extract_lecturers)
df_resolved['toimumisajad'] = df_resolved['toimumisajad_json'].apply(extract_schedule_days_et)

df_resolved['oppeaste_puhas'] = df_resolved['oppeaste'].apply(extract_text_values)

print("Samm 3: JSON töötlemine valmis.")
print(df_resolved[['aine_kood', 'oppejoud', 'toimumisajad', 'oppeaste_puhas']].head(3))

Ekstraheerime JSON väljadest infot...
Samm 3: JSON töötlemine valmis.
     aine_kood  \
0  OIEO.06.046   
2  KKSB.05.092   
3  ARKI.02.030   

                                                                             oppejoud  \
0                                                                           Gea Lepik   
2                                                                    Kadri Medijainen   
3  Jekaterina Nerman, Jürgen Rünk, Pille-Riin Värk, Ragnar Lõivukene, Viljo Kübarsepp   

         toimumisajad                              oppeaste_puhas  
0                 NaN                                 magistriõpe  
2  Neljapäev, Laupäev                             bakalaureuseõpe  
3                 NaN  integreeritud bakalaureuse- ja magistriõpe  


### Samm 4 (Valikuline): Info koondamine mitmest allikast

Kui vajalik info on mitmes veerus, saame need ühendada. Hetkel teeme seda näiteks hindamisinfoga.

In [4]:
grading_source_cols = [
    ('Miinimumnõuded', 'version__grading__grade_preconditions__et'),
    ('Hindamismeetod', 'version__grading__grade_evaluation__et'),
    ('Võlgnevuste likvideerimine', 'version__grading__debt_elimination__et'),
]

def combine_grading_info(row):
    parts = []
    for label, col in grading_source_cols:
        value = row.get(col)
        if pd.notna(value) and str(value).strip() != '':
            parts.append(f"{label}: {str(value).strip()}")
    return "\n".join(parts) if parts else np.nan

df_resolved['hindamine_info'] = df_resolved.apply(combine_grading_info, axis=1)
print("Samm 4: Hindamisinfo koondatud.")
print(df_resolved['hindamine_info'].notna().value_counts())

Samm 4: Hindamisinfo koondatud.
hindamine_info
True     2779
False     161
Name: count, dtype: int64


### Samm 5: Lõplik viimistlus

Valime lõplikud veerud, lisame hard-filtri tunnused ja valmistame RAG jaoks ühendatud tekstivälja.

In [5]:
# Hard-filtri abiveerud
def map_eristavus(value):
    if pd.isna(value):
        return np.nan
    text = str(value).strip().lower()
    if 'mitteerist' in text:
        return 'mitteeristav'
    if 'erist' in text:
        return 'eristav'
    return text


df_resolved['eristavus'] = df_resolved['hindamisskaala'].apply(map_eristavus)

final_cols = [
    'aine_kood', 'nimi_et', 'nimi_en', 'eap',
    'semester', 'oppekeel', 'oppeaste_puhas',
    'teaduskond', 'instituut', 'linn', 'eristavus',
    'kirjeldus', 'eesmargid', 'opivaljundid',
    'hindamine_info', 'toimumisajad', 'oppejoud'
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

# RAG jaoks ühendatud kirjeldav väli
rag_text_cols = [
    'nimi_et', 'aine_kood', 'eap', 'semester', 'oppekeel', 'oppeaste_puhas',
    'teaduskond', 'instituut', 'eristavus', 'kirjeldus',
    'eesmargid', 'opivaljundid', 'hindamine_info',
    'toimumisajad', 'oppejoud'
]
rag_text_cols = [c for c in rag_text_cols if c in df_final.columns]

def build_rag_text(row):
    parts = []
    for col in rag_text_cols:
        value = row.get(col)
        if pd.notna(value) and str(value).strip() != '':
            parts.append(f"{col}: {value}")
    return "\n".join(parts)

df_final['kirjeldus_kogu'] = df_final.apply(build_rag_text, axis=1)
df_final['kirjeldus_kogu_pikkus'] = df_final['kirjeldus_kogu'].str.len()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print("Samm 5: Lõplik andmestik koostatud.")

Lõplik ridade arv: 2940, lõplik veergude arv: 19
Samm 5: Lõplik andmestik koostatud.


### Samm 6: Salvestamine, kvaliteedikontroll ja esitamiseks vajalikud väljundid

Selles sammus:
- salvestame puhastatud faili;
- arvutame puuduvate väärtuste hulga igas veerus;
- kuvame kategooriliste tunnuste enim levinud väärtused;
- näitame aine "Sissejuhatus andmeteadusesse" puhastatud väljundi loetaval kujul;
- kuvame `kirjeldus_kogu` tähemärkide arvu statistika.

In [6]:
# Salvesta fail
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")

# 1) Puuduvate väärtuste hulk
print('\nPuuduvate väärtuste arv veergude kaupa:')
missing = df_final.isna().sum().sort_values(ascending=False)
display(missing)

# 2) Kategooriliste tunnuste enim levinud väärtused
print('\nKategooriliste tunnuste top-väärtused:')
for col in ['oppekeel', 'semester', 'oppeaste_puhas', 'eristavus', 'linn']:
    if col in df_final.columns:
        print(f"\nVeerg: {col}")
        display(df_final[col].value_counts(dropna=False).head(5))

# 3) "Sissejuhatus andmeteadusesse" aine ilus väljund
pattern_et = r'sissejuhatus\s+andmeteadus'
pattern_en = r'introduction\s+to\s+data\s+science'

mask_et = df_final['nimi_et'].fillna('').str.contains(pattern_et, case=False, regex=True)
mask_en = df_final['nimi_en'].fillna('').str.contains(pattern_en, case=False, regex=True)

sample_df = df_final.loc[mask_et | mask_en]

# fallback: kui filtritega kadus välja, otsime samast puhastusvoost enne lõppvalikut
if sample_df.empty and 'df_resolved' in globals():
    et_src = df_resolved['nimi_et'].fillna('').str.contains(pattern_et, case=False, regex=True)
    en_src = df_resolved['nimi_en'].fillna('').str.contains(pattern_en, case=False, regex=True)
    fallback = df_resolved.loc[et_src | en_src].copy()
    fallback_cols = [
        'aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester', 'oppekeel',
        'oppeaste_puhas', 'kirjeldus', 'hindamine_info', 'toimumisajad', 'oppejoud'
    ]
    fallback_cols = [c for c in fallback_cols if c in fallback.columns]
    if not fallback.empty:
        sample_df = fallback[fallback_cols]

if not sample_df.empty:
    sample_course = sample_df.head(1).T
    sample_course.columns = ['väärtus']
    print('\nNäidis: "Sissejuhatus andmeteadusesse" (puhastatud kuju)')
    display(sample_course)
else:
    print('\nAinet "Sissejuhatus andmeteadusesse" (või "Introduction to Data Science") ei leitud ka fallback-otsinguga.')

# 4) Kirjeldusvälja pikkuse statistika
print('\nTunnuse kirjeldus_kogu pikkuse statistika:')
display(df_final['kirjeldus_kogu_pikkus'].describe())

# 5) Kiire eelvaade
print('\nLõppandmestiku eelvaade:')
display(df_final.head(3))

Samm 6: Andmestik salvestatud faili '../andmed/andmed_aasta.csv'.

Puuduvate väärtuste arv veergude kaupa:


toimumisajad             1862
eesmargid                1610
opivaljundid             1610
hindamine_info            161
oppeaste_puhas             35
oppejoud                    1
aine_kood                   0
nimi_en                     0
nimi_et                     0
semester                    0
eap                         0
eristavus                   0
linn                        0
instituut                   0
teaduskond                  0
oppekeel                    0
kirjeldus                   0
kirjeldus_kogu              0
kirjeldus_kogu_pikkus       0
dtype: int64


Kategooriliste tunnuste top-väärtused:

Veerg: oppekeel


oppekeel
eesti keel      2172
inglise keel     768
Name: count, dtype: int64


Veerg: semester


...

,väärtus
aine_kood,SVMJ.04.015
nimi_et,Sissejuhatus andmeteadusse ja ärianalüütikasse
nimi_en,Introduction to Data Science and Business Analytics
eap,6.0
semester,kevad
oppekeel,eesti keel
oppeaste_puhas,bakalaureuseõpe
teaduskond,sotsiaalteaduste valdkond
instituut,majanduse modelleerimise õppetool
linn,Tartu linn



Tunnuse kirjeldus_kogu pikkuse statistika:


count    2940.000000
mean     1583.336054
std       778.769661
min       252.000000
25%      1016.000000
50%      1430.000000
75%      1974.000000
max      6011.000000
Name: kirjeldus_kogu_pikkus, dtype: float64


Lõppandmestiku eelvaade:


,aine_kood,nimi_et,nimi_en,eap,semester,oppekeel,oppeaste_puhas,teaduskond,instituut,linn,eristavus,kirjeldus,eesmargid,opivaljundid,hindamine_info,toimumisajad,oppejoud,kirjeldus_kogu,kirjeldus_kogu_pikkus
0,OIEO.06.046,Rahvusvaheline eraõigus,Private International Law,6.00,kevad,eesti keel,magistriõpe,sotsiaalteaduste valdkond,eraõiguse osakond,Tartu linn,eristav,"Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...",NaN,NaN,Miinimumnõuded: Aktiivne osalemine seminarides.\nHindamismeetod: 100% kirjalik eksam.\nVõlgnevus...,NaN,Gea Lepik,nimi_et: Rahvusvaheline eraõigus\naine_kood: OIEO.06.046\neap: 6.0\nsemester: kevad\noppekeel: e...,817
2,KKSB.05.092,Praktiseerimine töökeskkonnas,Practice in the Work Environment,3.00,kevad,eesti keel,bakalaureuseõpe,meditsiiniteaduste valdkond,sporditeaduste ja füsioteraapia instituut,Tartu linn,eristav,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,NaN,NaN,Miinimumnõuded: Nõutava tähtaegne esitamine\nVõlgnevuste likvideerimine: Võlgnevuste likvideerim...,"Neljapäev, Laupäev",Kadri Medijainen,nimi_et: Praktiseerimine töökeskkonnas\naine_kood: KKSB.05.092\neap: 3.0\nsemester: kevad\noppek...,637
3,ARKI.02.030,Lastekirurgia,Paediatric Surgery,2.25,sügis,inglise keel,integreeritud bakalaureuse- ja magistriõpe,meditsiiniteaduste valdkond,kirurgiakliinik,Tartu linn,eristav,"Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud aine m...",NaN,NaN,Miinimumnõuded: Participation in practical classes; max 1 absence\nHindamismeetod: A part of the...,NaN,"Jekaterina Nerman, Jürgen Rünk, Pille-Riin Värk, Ragnar Lõivukene, Viljo Kübarsepp",nimi_et: Lastekirurgia\naine_kood: ARKI.02.030\neap: 2.25\nsemester: sügis\noppekeel: inglise ke...,824
